<a href="https://colab.research.google.com/github/BilgehanUnal/BilgehanUnal/blob/main/Bilgehan_task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 8: Please make a joke recommender system using following Jester data set. You can leverage n8 notebook for this purpose.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Movie metadata
dfJoke = pd.read_csv('/content/drive/MyDrive/Data/task_8/JokeText.csv')

In [9]:
dfJoke.head()

,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


# **Content Based Filtering**


In [10]:

from sklearn.feature_extraction.text import TfidfVectorizer
# min_df should be an integer greater than 0, or a float between 0 (exclusive) and 1 (inclusive)
mdlTfvJoke = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english') # Changed min_df to 1
tfidf_matrix = mdlTfvJoke.fit_transform(dfJoke['JokeText'])
tfidf_matrix.shape

(100, 3774)

In [11]:
# Calculate cosine similarity between each pair of movies as a function of the similarity of the common terms

from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(100, 100)

In [12]:
dfJoke['JokeText'][80]

'An Asian man goes into a New York CityBank to exchange 10,000 yen for\nAmerican Currency.  The teller gives him $72.00.  The next month the\nAsian man goes into the same bank with 10,000 yen and receives $62.00.\nHe asks, "How come? Only $62.00?" The teller says "Fluctuations-\nFluctuations!"\n\nWhereupon the Asian man looks back at the teller and says "Fluk you\nAmelicans too!"\n'

Predict

In [14]:
# Prepare recommendation function (build code from scratch and then package as function for ease of understanding)

titles = dfJoke['JokeText']
# Index the series by 'JokeText' instead of 'JokeId'
indices = pd.Series(dfJoke.index, index=dfJoke['JokeId'])

def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


In [13]:
dfJoke['JokeText'][10]

"Q. What do a hurricane, a tornado, and a redneck\ndivorce all have in common? \nA. Someone's going to lose their trailer...\n"

In [15]:
get_recommendations(10).head(3)

51    Q: What do Monica Lewinsky and Bob Dole have i...
37    "May I take your order?" the waiter asked. \n\...
68    This guys wife asks, "Honey if I died would yo...
Name: JokeText, dtype: object

In [16]:
dfJoke['JokeText'][5]

"Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of\ntown. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and\nthe attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. \n\nAs Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and not him ? You could've\nbeen the wife of a grease monkey !' \n\nTo which Hillary replied, 'No, Bill. If I would have married him, you'd be pumping gas and he would be the President !' \n"

In [17]:
get_recommendations(5).head(3)

98    A bus station is where a bus stops.\nA train s...
79    Hillary, Bill Clinton and the Pope are sitting...
26    Clinton returns from a vacation in Arkansas an...
Name: JokeText, dtype: object

# **Collaborative Filtering**


In [18]:
# User ratings for each movie
dfRate1 = pd.read_csv('/content/drive/MyDrive/Data/task_8/UserRatings1.csv')

# User ratings for each movie
dfRate2 = pd.read_csv('/content/drive/MyDrive/Data/task_8/UserRatings2.csv')

In [19]:
dfRate1.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


In [20]:
import pandas as pd
dfRate1 = dfRate1.apply(lambda row: row.fillna(row.mean()), axis=1)
dfRate2 = dfRate2.apply(lambda row: row.fillna(row.mean()), axis=1)

In [21]:
missing_summary1 = dfRate1.isnull().sum().sum()
missing_summary2 = dfRate2.isnull().sum().sum()

In [22]:
missing_summary1

0

In [23]:
missing_summary2

0

In [24]:
dfRate1.head(5)


,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0.0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,0.899458,0.899458,0.899458,0.899458,0.899458,0.899458,0.899458,0.899458,2.910000,0.899458
1,1.0,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,0.184026,0.184026,0.184026,-5.630000,0.184026,-6.070000,0.184026,-1.600000,-4.560000,0.184026
2,2.0,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,0.193670,0.193670,0.193670,0.193670,0.193670,4.080000,0.193670,0.193670,8.980000,0.193670
3,3.0,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,-1.415871,-1.415871,-1.415871,-1.415871,-1.415871,-1.415871,-1.415871,-1.415871,-1.415871,-1.415871
4,4.0,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.280000,-0.490000,5.100000,-0.290000,-3.540000,-1.360000,7.480000,-5.780000,0.730000,2.620000


In [26]:
!pip install scikit-surprise

import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split



melted_dfRate1 = pd.melt(dfRate1, id_vars=['JokeId'], var_name='userId', value_name='rating')

melted_dfRate1 = melted_dfRate1.dropna(subset=['rating'])

reader = Reader(rating_scale=(0,5))
X = Dataset.load_from_df(melted_dfRate1[['userId', 'JokeId', 'rating']], reader)
X_train, X_test = train_test_split(X, test_size=.25)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357284 sha256=ca9c131d3cc75acea7c80b597d14eae5d22d56d71969d102fb294cce65d6fce6
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [27]:
melted_dfRate1

,JokeId,userId,rating
0,0.0,User1,5.100000
1,1.0,User1,4.900000
2,2.0,User1,1.750000
3,3.0,User1,-4.170000
4,4.0,User1,5.150000
...,...,...,...
3670995,95.0,User36710,1.521741
3670996,96.0,User36710,1.664735
3670997,97.0,User36710,0.935877
3670998,98.0,User36710,0.047825


# **Build Model**

In [28]:
# Define SVD model

from surprise import SVD

mdlSvdJokeRtg = SVD()

In [29]:
# Fit SVD model

mdlSvdJokeRtg.fit(X_train)
test_pred = mdlSvdJokeRtg.test(X_test)

In [30]:
# Evalute SVD accuracy

from surprise import accuracy

accuracy.rmse(test_pred)

RMSE: 4.2336


4.2336243433827825

In [31]:
from surprise.model_selection import GridSearchCV
from surprise import SVD

param_grid = {'n_epochs': [5, 10],
              'lr_all': [0.002, 0.005],
              'reg_all': [0.4]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=2)

gs.fit(X)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

4.298091848981153
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [32]:
# Cross-validate
from surprise.model_selection import cross_validate

cross_validate(mdlSvdJokeRtg, X, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    4.2410  4.2384  4.2349  4.2448  4.2435  4.2405  0.0036  
MAE (testset)     3.2086  3.2079  3.2057  3.2141  3.2147  3.2102  0.0036  
Fit time          59.93   62.78   60.16   61.60   62.96   61.49   1.27    
Test time         9.71    9.23    10.00   8.24    12.72   9.98    1.50    


{'test_rmse': array([4.24104576, 4.23841991, 4.23489082, 4.24480792, 4.24350293]),
 'test_mae': array([3.2086134 , 3.2078623 , 3.20566971, 3.21414993, 3.21466427]),
 'fit_time': (59.92964720726013,
  62.78404498100281,
  60.16235589981079,
  61.60326337814331,
  62.957990407943726),
 'test_time': (9.714044570922852,
  9.232802391052246,
  10.000315427780151,
  8.237912654876709,
  12.720933198928833)}

# **Predict**

In [46]:
#lets use the model that we created

User='User20'
Joke= 10
rated_jokes = melted_dfRate1[melted_dfRate1['userId'] == User]['JokeId'].unique()

if rated_jokes.size > 0:
    predict_value_of_joke = rated_jokes[Joke]

    prediction = mdlSvdJokeRtg.predict(User, predict_value_of_joke)
    print(f"models prediction of joke {predict_value_of_joke} evaluated by user {User} is {prediction.est}")

models prediction of joke 10.0 evaluated by user User20 is 2.6076326370711587


In [47]:
mdlSvdJokeRtg.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=0.7984198365426585, details={'was_impossible': False})